In [1]:
import zipfile
import os
import pandas as pd

In [27]:
def concatenar_csvs(diretorio):
    frames = []

    for arquivo in os.listdir(diretorio):
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(diretorio, arquivo)
            
            df = pd.read_csv(caminho_arquivo, encoding='utf-8', delimiter=";", decimal=",")
            df = df.rename(columns={df.columns[0]: "data_base"})
            df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format="%Y-%m-%d")  #2018 tem duas colunas chamadas de data_base, por isso precisamos renomear o nome das colunas
            df = df.iloc[:, [0] + list(range(13, 23))]  
            df = df.groupby(df.iloc[:, 0]).apply(lambda x: x.iloc[:, 1:].sum(axis=0)).reset_index(drop=False)
            
            frames.append(df)

    df_concatenado = pd.concat(frames, ignore_index=True)

    return df_concatenado

In [28]:
anos = list(range(2012, 2024))
dataframes = []

for ano in anos:
    diretorio = f"planilha_{ano}"
    dataframe_ano = concatenar_csvs(diretorio)
    dataframes.append(dataframe_ano)

df_total = pd.concat(dataframes, ignore_index=False)

In [37]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135 entries, 0 to 7
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   data_base                       135 non-null    datetime64[ns]
 1   a_vencer_ate_90_dias            135 non-null    float64       
 2   a_vencer_de_91_ate_360_dias     135 non-null    float64       
 3   a_vencer_de_361_ate_1080_dias   135 non-null    float64       
 4   a_vencer_de_1081_ate_1800_dias  135 non-null    float64       
 5   a_vencer_de_1801_ate_5400_dias  135 non-null    float64       
 6   a_vencer_acima_de_5400_dias     135 non-null    float64       
 7   vencido_acima_de_15_dias        135 non-null    float64       
 8   carteira_ativa                  135 non-null    float64       
 9   carteira_inadimplida_arrastada  135 non-null    float64       
 10  ativo_problematico              135 non-null    float64       
dtypes: datetime64

In [35]:
df_total.to_csv("analise_divida_tempo.csv", index = False)

In [1]:
fig = go.Figure()

for col in df_total.columns[1:]:
    if col != 'carteira_ativa':
        fig.add_trace(go.Scatter(x=df_total['data_base'], 
                                 y=df_total[col], 
                                 mode='lines', 
                                 name=col,
                                yaxis='y2'))

fig.add_trace(go.Bar(x=df_total['data_base'], 
                     y=df_total['carteira_ativa'],
                    opacity=0.5,
                    showlegend=False))

fig.update_layout(
    title='Parcelas a vencer vs Carteira ativa',
    xaxis_title='Data',
    yaxis_title='Parcelas das operações de crédito',
    yaxis2=dict(
        title='Carteira ativa',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        y=-0.2,
        traceorder='normal',
        orientation='h',
        font=dict(
            size=12,
            
        ),
    ),
    template='seaborn'
)

st.plotly_chart(fig, use_container_width=True, height=200)

NameError: name 'df_total' is not defined